# Teoría

## ¿Qué es Web Scraping?

Web Scraping es una técnica en la cuál un programa de computadora **extrae datos** desde un sitio web. Dichos programas son llamados **Bots** o **Web crawlers**. Escencialmente se busca copiar datos de una web en una **base de datos** o una **hoja de cálculo**, para posteriormente visualizarlos o analizarlos.

## ¿Por qué y cuándo aplicaríamos Web Scraping?

- El principal motivo para utilizar Web Scraping es la necesidad de acceder a contenido **público** de la web, al cual no podemos acceder mediante una **API** (Application Programming Interface). 

- Podemos aplicar Web Scraping cuando la página nos lo permita, para verificar el permiso podemos ver el archivo robots.txt de la página objetivo.

## Precauciones a tener en cuenta 🤗🤝

- Debemos hacernos una idea de la capacidad de la web en la que vamos a realizar Web Scraping para tratar de no **desbordar sus servidores**. 

- Tambien es necesario que verifiquemos si hay algun dato sensible que debamos reportar/tratar de manera especial. 

## Recursos adicionales

- [Apify](https://apify.com/)
- [Datos Abiertos Gobierno Peruano](https://www.datosabiertos.gob.pe//)
- [Conjuntos de datos humanitarios](https://competent-hopper-e1121e.netlify.app/)


# Live Coding - Caso 1: Wikipedia <img src="https://img.icons8.com/color/96/000000/wikipedia-logo.png"/>

## 1. Identificación de datos

**Objetivo**: Quiero analizar la selección de Fútbol de Perú

¡Ajá, puedo obtener datos de las tablas de Wikipedia! Para este ejemplo veamos la sección "Últimos Convocados".

In [1]:
# Importamos las librerias
import bs4 as bs
import requests
import pandas as pd
from google.colab import files

# Definamos la URL
URL = "https://es.wikipedia.org/wiki/Selecci%C3%B3n_de_f%C3%BAtbol_del_Per%C3%BA"

# Extraemos el codigo fuente (HTML)
source = requests.get(URL).text
soup = bs.BeautifulSoup(source,'lxml')

## 2. Recopilando la información necesaria

In [2]:
# title of the page
print(soup.title, end='\n\n')

<title>Selección de fútbol del Perú - Wikipedia, la enciclopedia libre</title>



In [3]:
# get attributes:
print(soup.title.name, end='\n\n')

title



In [4]:
# get values:
print(soup.title.string, end='\n\n')

Selección de fútbol del Perú - Wikipedia, la enciclopedia libre



In [5]:
# beginning navigation:
print(soup.title.parent.name, end='\n\n')

head



In [6]:
# getting specific values:
print(soup.p, end='\n\n')

<p>La <b>selección de fútbol del <a href="/wiki/Per%C3%BA" title="Perú">Perú</a></b> es el equipo representativo de dicho país en las competiciones oficiales de <a href="/wiki/F%C3%BAtbol" title="Fútbol">fútbol</a> <a href="/wiki/Var%C3%B3n" title="Varón">masculino</a>. Su organización está a cargo de la <a href="/wiki/Federaci%C3%B3n_Peruana_de_F%C3%BAtbol" title="Federación Peruana de Fútbol">Federación Peruana de Fútbol</a> (FPF), la cual es una de las diez federaciones miembro de la <a href="/wiki/Confederaci%C3%B3n_Sudamericana_de_F%C3%BAtbol" title="Confederación Sudamericana de Fútbol">Confederación Sudamericana de Fútbol (Conmebol)</a>. Su debut se produjo el <a href="/wiki/1_de_noviembre" title="1 de noviembre">1 de noviembre</a> de <a href="/wiki/1927" title="1927">1927</a> ante la selección de <a href="/wiki/Selecci%C3%B3n_de_f%C3%BAtbol_de_Uruguay" title="Selección de fútbol de Uruguay">Uruguay</a> en el <a href="/wiki/Campeonato_Sudamericano_1927" title="Campeonato Sudamer


## 3. Disenando el Web Scraper

1. Identifiquemos la tabla objetivo: "Ultimos Convocados"
2. Seleccionar la tabla objetivo - Tag name, XPath u otro
3. Extraer nombres de columnas - Sólo **1** vez
4. Extraer datos de cada fila - Iterar sobre las filas
5. Guardar archivo en formato CSV (Comma Separated Values)


## 4. Implementación del Web Scraper


In [7]:
tables = soup.find_all('table')

In [8]:
type(tables)

bs4.element.ResultSet

In [9]:
len(tables)

26

- **find_all('tr')**: para identificar solo las filas con contenido

In [10]:
table_rows = tables[12].find_all('tr')

In [11]:
len(table_rows)

32

- **find_all('th')**: para identificar solo los nombres de las columnas

In [12]:
column_names = [th.text.strip('\n') for th in table_rows[0].find_all('th')] #con strip dividimos el objeto en unidades
column_names

['', 'Nombre', 'Pos.', 'Edad', 'PJ', 'Goles', 'Equipo', 'Formativo']

In [13]:
column_names[0] = 'Num.' #Reenplazamos el vacío por Num
column_names

['Num.', 'Nombre', 'Pos.', 'Edad', 'PJ', 'Goles', 'Equipo', 'Formativo']

 - Generamos una **función** para ejecutar el WebScraper y guardar el resultado en **data**

In [14]:
# Extraer datos de la tabla
data = []

# Iterar sobre cada fila
for tr in table_rows[1:-1]:
  row_data = []

  # Valor especial numero de camiseta
  num = tr.find('th').text.strip('\n')
  row_data.append(num)

  # Extraer los demas valores de la fila
  for td in tr.find_all('td'):
    value = td.text.strip('\n')

    # Valor especial de posicion de la cancha (img (href) dentro de un enlace (a))
    if value == '':
      value = td.find('a')['href']

    row_data.append(value)

  # Almacenar los datos de la fila
  print(row_data, end='\n\n')
  data.append(row_data)

['1', 'Pedro Gallese', '/wiki/Archivo:FootballPositionGK_es.png', '30\xa0años', '62', '-64', ' Orlando City S.C.', ' U. San Martín']

['12', 'José Carvallo', '/wiki/Archivo:FootballPositionGK_es.png', '34\xa0años', '7', '-4', ' Universitario', ' Universitario']

['21', 'Carlos Cáceda', '/wiki/Archivo:FootballPositionGK_es.png', '28\xa0años', '6', '-3', ' F.B.C. Melgar', ' Universitario']

['2', 'Luis Abram', '/wiki/Archivo:FootballPositionCT_es.png', '24\xa0años', '20', '1', ' Vélez Sarfield', ' Regatas']

['3', 'Aldo Corzo', '/wiki/Archivo:FootballPositionCT_es.png', '31\xa0años', '31', '0', ' Universitario', ' Regatas']

['4', 'Anderson Santamaría', '/wiki/Archivo:FootballPositionCT_es.png', '28\xa0años', '17', '0', ' Atlas F.C.', ' Deportivo Municipal']

['5', 'Christian Ramos', '/wiki/Archivo:FootballPositionCT_es.png', '31\xa0años', '74', '3', ' U. César Vallejo', ' Sporting Cristal']

['6', 'Miguel Trauco', '/wiki/Archivo:FootballPositionCT_es.png', '27 años', '50', '0', ' A.S. S

- Generamos una **tabla** usando **pd.Dataframe** y guardamos el resultado en **convocados_peru_df**

In [15]:
# Creamos un DataFrame / Tabla de datos
convocados_peru_df = pd.DataFrame(data, columns=column_names)
convocados_peru_df

,Num.,Nombre,Pos.,Edad,PJ,Goles,Equipo,Formativo
0,1,Pedro Gallese,/wiki/Archivo:FootballPositionGK_es.png,30 años,62,-64,Orlando City S.C.,U. San Martín
1,12,José Carvallo,/wiki/Archivo:FootballPositionGK_es.png,34 años,7,-4,Universitario,Universitario
2,21,Carlos Cáceda,/wiki/Archivo:FootballPositionGK_es.png,28 años,6,-3,F.B.C. Melgar,Universitario
3,2,Luis Abram,/wiki/Archivo:FootballPositionCT_es.png,24 años,20,1,Vélez Sarfield,Regatas
4,3,Aldo Corzo,/wiki/Archivo:FootballPositionCT_es.png,31 años,31,0,Universitario,Regatas
5,4,Anderson Santamaría,/wiki/Archivo:FootballPositionCT_es.png,28 años,17,0,Atlas F.C.,Deportivo Municipal
6,5,Christian Ramos,/wiki/Archivo:FootballPositionCT_es.png,31 años,74,3,U. César Vallejo,Sporting Cristal
7,6,Miguel Trauco,/wiki/Archivo:FootballPositionCT_es.png,27 años,50,0,A.S. Saint-Étienne,Virgen de Chapi
8,15,Carlos Zambrano,/wiki/Archivo:FootballPositionCT_es.png,31 años,50,4,Boca Juniors,FC Schalke 04
9,17,Luis Advíncula,/wiki/Archivo:FootballPositionCT_es.png,29 años,89,1,Rayo Vallecano,Esther Grande


- Vamos a **reemplazar** la columna **Pos.** para extraer la posición del jugador de todo el texto.

In [16]:
valores_posicion = {
  '/wiki/Archivo:FootballPositionGK_es.png': 'Arquero',
  '/wiki/Archivo:FootballPositionCT_es.png': 'Defensa',
  '/wiki/Archivo:FootballPositionMID_es.png': 'Volante',
  '/wiki/Archivo:FootballPositionFWD_es.png': 'Delantero'
}

convocados_peru_df['Pos.'] = convocados_peru_df['Pos.'].replace(valores_posicion)
convocados_peru_df

,Num.,Nombre,Pos.,Edad,PJ,Goles,Equipo,Formativo
0,1,Pedro Gallese,Arquero,30 años,62,-64,Orlando City S.C.,U. San Martín
1,12,José Carvallo,Arquero,34 años,7,-4,Universitario,Universitario
2,21,Carlos Cáceda,Arquero,28 años,6,-3,F.B.C. Melgar,Universitario
3,2,Luis Abram,Defensa,24 años,20,1,Vélez Sarfield,Regatas
4,3,Aldo Corzo,Defensa,31 años,31,0,Universitario,Regatas
5,4,Anderson Santamaría,Defensa,28 años,17,0,Atlas F.C.,Deportivo Municipal
6,5,Christian Ramos,Defensa,31 años,74,3,U. César Vallejo,Sporting Cristal
7,6,Miguel Trauco,Defensa,27 años,50,0,A.S. Saint-Étienne,Virgen de Chapi
8,15,Carlos Zambrano,Defensa,31 años,50,4,Boca Juniors,FC Schalke 04
9,17,Luis Advíncula,Defensa,29 años,89,1,Rayo Vallecano,Esther Grande


- Tranformamos algunos **tipos de datos** a **entero**

In [17]:
# Convierte a valores de tipo entero (int)
convocados_peru_df['PJ'].astype(int)
convocados_peru_df['Goles'].astype(int)

0    -64
1     -4
2     -3
3      1
4      0
5      0
6      3
7      0
8      4
9      1
10     1
11     0
12     0
13     0
14    10
15     3
16     2
17     3
18     3
19     0
20     0
21     0
22     0
23    27
24     4
25     1
26     6
27    13
28     2
29     0
Name: Goles, dtype: int64

In [18]:
# Eliminando el texto 'años' y convirtiendo el valor atipo de dato entero (int)
convocados_peru_df['Edad'] = convocados_peru_df['Edad'].str.strip(' años').astype(int)

- Finalmente generamos y descargamos el archivo **csv**

In [19]:
# Grabamos el archivo CSV dentro de Colab
convocados_peru_df.to_csv('convocados_peru.csv', index=False)

In [20]:
# Descargamos el archivo en nuestra PC
files.download('convocados_peru.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Caso 2: Captura de datos de Retail

## 1. Identificación de datos

**Objetivo**: Obtener precios de productos en Supermercados (Wong)

¡Ajá, puedo obtener datos desde su pagina web! Para este ejemplo veamos la sección "Abarrotes".

## 3. Disenando el Web Scraper

1. Identifiquemos la items objetivo: "Productos dentro de la cuadricula de productos"
2. Seleccionar el conjunto de los items objetivo (ul) - Tag name, XPath u otro
3. Definir nombres de columnas
4. Extraer datos de cada item (li) - Iterar sobre los items (li)
5. Guardar archivo en formato CSV (Comma Separated Values)

In [21]:
# Definamos la URL
WONG_URL = "https://www.wong.pe/abarrotes"

# Extraemos el codigo fuente (HTML)
source = requests.get(WONG_URL).text
soup = bs.BeautifulSoup(source,'lxml')

In [22]:
product_lists = soup.find_all('ul')

In [23]:
len(product_lists)

56

In [24]:
product_items = product_lists[18].find_all('li')

In [25]:
len(product_items)

73

In [26]:
selected_div_attrs = ['data-name', 'data-price', 'data-brand', 'data-category']
data = []
for product_item in product_items:
  try:
    product_main_div = product_item.find('div')
    product_data = [product_main_div[attr] for attr in selected_div_attrs]
    print(product_data, end='\n\n')
    data.append(product_data)
  except:
    pass

['Aceite Vegetal Primor Premium Botella 1 L', 'S/. 7.49', 'Primor', 'https://www.wong.pe/abarrotes/aceites/aceites-vegetales']

['Papas Artesanales Andinas Saladas Inka Chips Bolsa 142 g', 'S/. 6.20', 'Inka Chips', 'https://www.wong.pe/abarrotes/galletas-snacks-y-golosinas/piqueos']

['Sal de Mesa Marina Emsal Bolsa 1 kg', 'S/. 2.15', 'Emsal', 'https://www.wong.pe/abarrotes/condimentos-vinagres-y-comida-instantanea/sal-sal-parrillera-y-sal-especial']

['Galletas Oreo Nabisco Original Pack 6 Unid x 36 g', 'S/. 3.29', 'Nabisco', 'https://www.wong.pe/abarrotes/galletas-snacks-y-golosinas/galletas']

['Galletas Vainilla Field Pack 6 Unid x 37 g', 'S/. 3.20', 'Field', 'https://www.wong.pe/abarrotes/galletas-snacks-y-golosinas/galletas']

['Galletas San Jorge Soda Pack 7 Unidades', 'S/. 2.89', 'San Jorge', 'https://www.wong.pe/abarrotes/galletas-snacks-y-golosinas/galletas']

['Galletas Morochas Nestlé Pack 6 Unidades', 'S/. 4.30', 'Nestlé', 'https://www.wong.pe/abarrotes/galletas-snacks-y-g

In [27]:
# Creamos un DataFrame / Tabla de datos
productos_wong_df = pd.DataFrame(data, columns=selected_div_attrs)

In [28]:
productos_wong_df

,data-name,data-price,data-brand,data-category
0,Aceite Vegetal Primor Premium Botella 1 L,S/. 7.49,Primor,https://www.wong.pe/abarrotes/aceites/aceites-...
1,Papas Artesanales Andinas Saladas Inka Chips B...,S/. 6.20,Inka Chips,https://www.wong.pe/abarrotes/galletas-snacks-...
2,Sal de Mesa Marina Emsal Bolsa 1 kg,S/. 2.15,Emsal,https://www.wong.pe/abarrotes/condimentos-vina...
3,Galletas Oreo Nabisco Original Pack 6 Unid x 36 g,S/. 3.29,Nabisco,https://www.wong.pe/abarrotes/galletas-snacks-...
4,Galletas Vainilla Field Pack 6 Unid x 37 g,S/. 3.20,Field,https://www.wong.pe/abarrotes/galletas-snacks-...
5,Galletas San Jorge Soda Pack 7 Unidades,S/. 2.89,San Jorge,https://www.wong.pe/abarrotes/galletas-snacks-...
6,Galletas Morochas Nestlé Pack 6 Unidades,S/. 4.30,Nestlé,https://www.wong.pe/abarrotes/galletas-snacks-...
7,Galletas Soda Field Pack 6 Unid x 34 g,S/. 2.30,Field,https://www.wong.pe/abarrotes/galletas-snacks-...
8,Spaguetti Don Vittorio Paquete 1 Kg,S/. 4.59,Don Vittorio,https://www.wong.pe/abarrotes/fideos-pastas-y-...
9,Lomos de Atún Campomar en Aceite Lata 170 gr,S/. 5.20,Campomar,https://www.wong.pe/abarrotes/alimentos-en-con...


In [29]:
# Grabamos el archivo CSV dentro de Colab
productos_wong_df.to_csv('productos_wong.csv', index=False)

In [30]:
# Descargamos el archivo en nuestra PC
files.download('productos_wong.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Recursos Adicionales de Beautiful Soup 4

- [Web scraping and parsing with Beautiful Soup 4](https://pythonprogramming.net/introduction-scraping-parsing-beautiful-soup-tutorial/)
- [Beautiful Soup Documentation¶](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)